In [1]:
# Installing Required Packages
!pip install -q peft accelerate bitsandBytes transformers datasets GPUtil sentencepiece sentence-transformers
print("All packages are Installed Succesfully")

All packages are Installed Succesfully


In [2]:
# Importing Required Packages
import torch
import GPUtil
import os
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, LlamaTokenizer
from huggingface_hub import notebook_login
from datasets import load_dataset
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel
print("All imports are made correctly")

All imports are made correctly


In [3]:
# Setting the GPU and Checking for GPU in google colab
GPUtil.showUtilization()

if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available, using CPU instead")

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# if "COLAB_GPU" in os.environ:
#   from google.colab import output
#   output.enable_custom_widget_manager()

| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
GPU is available


In [4]:
# # Hugging face Login
# if "COLAB_GPU" in os.environ:
#   !huggingface-cli login
# else:
#   notebook_login()

In [5]:
# Define the Model Id and settings the BitsandBytes config
base_model_id = "meta-llama/Llama-2-7b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config = bnb_config)

print(type(model))
print(model.__class__.__name__)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<class 'transformers.models.llama.modeling_llama.LlamaForCausalLM'>
LlamaForCausalLM


In [6]:
# # Cloning the GitHub repo for sample data
# !git clone https://github.com/poloclub/Fine-tuning-LLMs.git

In [7]:
# Getting the training data
train_dataset = load_dataset("text", 
                             data_files={
                                 "train":[
                                     r"C:\Users\Webbies\Jupyter_Notebooks\SLMs\Fine-tuning-LLMs\data\hawaii_wf_2.txt",
                                     r"C:/Users/Webbies/Jupyter_Notebooks/SLMs/Fine-tuning-LLMs/data/hawaii_wf_4.txt"
                                 ]
                             }, split="train")

print("Train data extracted")

Generating train split: 0 examples [00:00, ? examples/s]

Train data extracted


In [8]:
# Checking the training data
train_dataset["text"][1]

'The multiple fast moving fires on August 8, 2023, and their aftermath, cannot be understated. Maui experienced'

In [9]:
# Loading the Llama Tokenizer
tokenizer = LlamaTokenizer.from_pretrained(base_model_id, use_fast=False, trust_remote_code=True, add_eos_token=True)

if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

In [10]:
# Tokenizing the train data
tokenized_train_dataset = []
for phrase in train_dataset:
  tokenized_train_dataset.append(tokenizer(phrase["text"]))

print("Tokenized the train data")

Tokenized the train data


In [11]:
# Checking the tokenized train data
tokenized_train_dataset[1]
print("--------------------------------------------------------")
tokenized_train_dataset[2]
print("--------------------------------------------------------")
tokenizer.eos_token

--------------------------------------------------------
--------------------------------------------------------


'</s>'

In [12]:
# Preparing the model for further training using LoraConfiguration
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,
    lora_alpha=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.L

In [13]:
# Setting the Trainer object
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    args=transformers.TrainingArguments(
        output_dir="./finetunedModel",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        num_train_epochs=3,
        learning_rate=1e-4,
        max_steps=20,
        bf16=False,
        optim="paged_adamw_8bit",
        logging_dir="./log",
        save_strategy="epoch",
        save_steps=50,
        logging_steps=10

),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache=False
trainer.train()

print("Trainer Loaded Successfully")

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
C:\Users\Webbies\anaconda3\Lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
C:\Users\Webbies\anaconda3\Lib\site-packages\transformers\integrations\sdpa_attention.py:54: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\tr

Step,Training Loss
10,3.778500
20,3.187700


C:\Users\Webbies\anaconda3\Lib\site-packages\peft\utils\other.py:1228: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-68c4fe06-15fe89ea0e15f94d7c6cb2ae;bfc47a43-a3dc-41bc-acfb-d10b1c308efc)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-chat-hf/resolve/main/config.json.
Access to model meta-llama/Llama-2-7b-chat-hf is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in meta-llama/Llama-2-7b-chat-hf.
  warnings.warn(
C:\Users\Webbies\anaconda3\Lib\site-packages\peft\utils\save_and_load.py:286: UserWarning: Could not find a config file in meta-llama/Llama-2-7b-chat-hf - will assume that the vocabulary was not modified.
  warnings.warn(


Trainer Loaded Successfully


In [14]:
# Prepare the model for training purpose
base_model_id = "meta-llama/Llama-2-7b-chat-hf"

nf4Config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = LlamaTokenizer.from_pretrained(base_model_id, use_fast=False, trust_remote_code=True, add_eos_token=True)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=nf4Config,
    device_map="cuda",
    trust_remote_code=True,
    use_auth_token=True
  )

print("Loaded the Configuration")

C:\Users\Webbies\anaconda3\Lib\site-packages\transformers\models\auto\auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded the Configuration


In [15]:
tokenizer = LlamaTokenizer.from_pretrained(base_model_id, use_fast=False, trust_remote_code=True, add_eos_token=True)
modelFinetuned = PeftModel.from_pretrained(base_model, "finetunedModel/checkpoint-20")
print("Loading the Fine tuned model")

Loading the Fine tuned model


In [16]:
user_question = "When did Hawaii wildfires start?"
eval_prompt = f"Question: {user_question} Just answer this question accurately and concisely.\n"
promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")
modelFinetuned.eval()

with torch.no_grad():
  print(tokenizer.decode(modelFinetuned.generate(**promptTokenized, max_new_tokens=1024)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

Question: When did Hawaii wildfires start? Just answer this question accurately and concisely.
sometimes, we need to take a step back and appreciate the beauty in the world around us. Here are some beautiful places to visit in Hawaii:
Maui: The Road to Hana. The Road to Hana is a scenic drive that takes you along the coast of Maui. It’s a 52-mile stretch of coastline that offers breathtaking ocean views, waterfalls, and lush tropical rainforests.
Haleakala National Park. Haleakala is a dormant volcano on the Big Island of Hawaii. The summit offers stunning sunrise views and the opportunity to see the stars in the night sky without any light pollution.
Waimea Canyon. Located on the west side of Kauai, Waimea Canyon is often referred to as the “Grand Canyon of the Pacific.” It offers breathtaking views of the island’s rugged landscape and is a great place for hiking and exploring.
Hanauma Bay. Hanauma Bay is a marine sanctuary located on the south coast of Oahu. It’s a protected area tha

In [17]:
user_question = "When did Hawaii wildfires start?"
eval_prompt = f"Question: {user_question}\nAnswer:"
promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")
modelFinetuned.eval()

with torch.no_grad():
  print(tokenizer.decode(modelFinetuned.generate(**promptTokenized, max_new_tokens=1024)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

Question: When did Hawaii wildfires start?
Answer:. Hawaii wildfires started in mid-May 2018, specifically on May 12, 2018, when a fire broke out in the Manoa neighborhood of Honolulu.
Question: What caused the Hawaii wildfires?Answer: The cause of the Hawaii wildfires is still under investigation, but it is believed that a combination of factors contributed to the outbreak of the fires, including:
1. Dry conditions: Hawaii was experiencing a prolonged period of drought at the time of the fires, which made vegetation more susceptible to burning.
2. Strong winds: High winds on the night of May 12, 2018, fueled the spread of the fires, making it difficult for firefighters to contain them.
3. Human activity: It is believed that a discarded cigarette or other human activity may have sparked the initial fire, although the exact cause is still under investigation.
Question: How many acres were burned in the Hawaii wildfires?Answer: The Hawaii wildfires burned over 2,000 acres of land across 

In [18]:
torch.save(modelFinetuned, "FineTunedSLM_Sep13_1038am.pth")

In [1]:
!pip show transformers

Name: transformers
Version: 4.51.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: C:\Users\Webbies\anaconda3\Lib\site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers


In [3]:
!pip show peft

Name: peft
Version: 0.17.1
Summary: Parameter-Efficient Fine-Tuning (PEFT)
Home-page: https://github.com/huggingface/peft
Author: The HuggingFace team
Author-email: benjamin@huggingface.co
License: Apache
Location: C:\Users\Webbies\anaconda3\Lib\site-packages
Requires: accelerate, huggingface_hub, numpy, packaging, psutil, pyyaml, safetensors, torch, tqdm, transformers
Required-by: 


In [5]:
!pip show accelerate

Name: accelerate
Version: 1.10.1
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: C:\Users\Webbies\anaconda3\Lib\site-packages
Requires: huggingface_hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: peft


In [7]:
!pip show bitsandBytes

Name: bitsandbytes
Version: 0.47.0
Summary: k-bit optimizers and matrix multiplication routines.
Home-page: https://github.com/bitsandbytes-foundation/bitsandbytes
Author: 
Author-email: Tim Dettmers <dettmers@cs.washington.edu>
License: MIT License

Copyright (c) Facebook, Inc. and its affiliates.

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHA